In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
df = pd.read_csv('train.txt',sep=';',header=None,names=['text','emotion'])
 #agr file mein header nhi hoga to pehle row ko pandas header smajh leta hai 
#name se hmlog khud column ke naam de rhe hain



        text        emotion
0   I love coding    happy
1   I feel tired     sad


In [13]:
df.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [14]:
df.isnull().sum()

text       0
emotion    0
dtype: int64

In [15]:
unique_emotions = df['emotion'].unique()
print(unique_emotions)
emotion_numbers = {}
i=0
for emo in unique_emotions:
    emotion_numbers[emo] = i;
    i=i+1
df['emotion'] = df['emotion'].map(emotion_numbers)
df

['sadness' 'anger' 'love' 'surprise' 'fear' 'joy']


,text,emotion
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,5
15998,i feel like this was such a rude comment and i...,1


In [16]:
df['text'] = df['text'].apply(lambda x: x.lower())


In [17]:
import string
def remove_punc(txt):
    return txt.translate(str.maketrans('','',string.punctuation))

# str.maketrans(from, to, delete) --maketrans() ek translation table banata hai
#string.punctuation  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
#Ye function translation table apply karta hai

In [18]:
df['text'] = df['text'].apply(remove_punc)


In [19]:
def remove_numbers(txt):
    new=""
    for i in txt:
        if not i.isdigit():
            new = new + i;
    return new
df['text'] = df['text'].apply(remove_numbers)


In [20]:
def remove_emojis(txt):
    new=""
    for i in txt:
        if i.isascii():
            new = new + i
    return new
df['text'] = df['text'].apply(remove_emojis)

In [21]:
%pip install nltk


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
stop_words = ENGLISH_STOP_WORDS
stop_words
#english lang ke stopwords ka set

KeyboardInterrupt: 

In [ ]:

df.loc[0]['text']


'i didnt feel humiliated'

In [ ]:
def remove(txt):
    if not isinstance(txt, str):
        return txt #check krta hai txt string hai ya nhi 
    words = txt.lower().split()
    cleaned = []
    for i in words:
        if i not in stop_words:
            cleaned.append(i)
    return ' '.join(cleaned)



In [ ]:
df['text'] = df['text'].apply(remove)
df.loc[1]['text']

'feeling hopeless damned hopeful just cares awake'

In [ ]:
df.head()

,text,emotion
0,didnt feel humiliated,0
1,feeling hopeless damned hopeful just cares awake,0
2,im grabbing minute post feel greedy wrong,1
3,feeling nostalgic fireplace know property,2
4,feeling grouchy,1


In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['emotion'], test_size=0.20, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer #text ko number mein change krne ke liye
#ml text nhi smjhta

from sklearn.naive_bayes import MultinomialNB
# Text classification ke liye best & fast

from sklearn.metrics import accuracy_score
#performance measure krne ke liye



In [ ]:

bow_vectorizer = CountVectorizer()
bow_vectorizer



,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (strip_accents and lowercase) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None
,"stop_words stop_words: {'english'}, list, default=NoneIf 'english', a built-in stop word list for English is used.There are several known issues with 'english' and you shouldconsider an alternative (see :ref:`stop_words`).If a list, that list is assumed to contain stop words, all of whichwill be removed from the resulting tokens.Only applies if ``analyzer == 'word'``.If None, no stop words will be used. In this case, setting `max_df`to a higher value, such as in the range (0.7, 1.0), can automatically detectand filter stop words based on intra corpus document frequency of terms.",None
,"token_pattern token_pattern: str or None, default=r""(?u)\\b\\w\\w+\\b""Regular expression denoting what constitutes a ""token"", only usedif ``analyzer == 'word'``. The default regexp select tokens of 2or more alphanumeric characters (punctuation is completely ignoredand always treated as a token separator).If there is a capturing group in token_pattern then thecaptured group content, not the entire match, becomes the token.At most one capturing group is permitted.",'(?u)\\b\\w\\w+\\b'
,"ngram_range ngram_range: tuple (min_n, max_n), default=(1, 1)The lower and upper boundary of the range of n-values for differentword n-grams or char n-grams to be extracted. All values of n suchsuch that min_n <= n <= max_n will be used. For example an``ngram_range`` of ``(1, 1)`` means only unigrams, ``(1, 2)`` meansunigrams and bigrams, and ``(2, 2)`` means only bigrams.Only applies if ``analyzer`` is not callable.","(1, ...)"
,"analyzer analyzer: {'word', 'char', 'char_wb'} or callable, default='word'Whether the feature should be made of word n-gram or charactern-grams.Option 'char_wb' creates character n-grams only from text insideword boundaries; n-grams at the edges of words are padded with space.If a callable is passed it is used to extract the sequence of featuresout of the raw, unprocessed input... versionchanged:: 0.21Since v0.21, if ``input`` is ``filename`` or ``file``, the data isfirst read from the file and then passed to the given callableanalyzer.",'word'


Sentences :
"I feel happy"
"I feel sad"

1. BoW pehle saare unique words nikalta hai:
["i", "feel", "happy", "sad"] -> Vocabulary (unique words)

2.Counts words in each sentence
sentence 1: "I feel happy"

Word	Count
i	    1
feel	1
happy	1
sad	    0

➡️ Vector:
[1, 1, 1, 0]


Sentence 2: "I feel sad"
Word	Count
i	    1
feel	1
happy	0
sad	    1

➡️ Vector:
[1, 1, 0, 1]

🎯 Final Bag of Words Matrix
Sentence	    i	feel	happy	sad
I feel happy	1	 1	     1	    0
I feel sad	    1	 1	     0	    1

This is known as Bag Of Words

In [ ]:
X_train_bow = bow_vectorizer.fit_transform(X_train)
# fit → training data se vocabulary seekhta
# transform → sentences ko number matrix banata

X_test_bow = bow_vectorizer.transform(X_test)
#data leak hojayega agr fit lagyenge to 

nb_model = MultinomialNB()  #text(count)
#multinomial mtlb words ki frequency ka count rakhna
# Gaussian nb continuos data ke liye hot hai

nb_model.fit(X_train_bow,y_train)

# Model learn karta hai:
# kaunse words kis emotion ke saath zyada aate hain
# "sad" → sadness
# "angry" → anger
# "love" → love

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [ ]:
pred_bow = nb_model.predict(X_test_bow)
print(accuracy_score(y_test, pred_bow))

0.76875


In [ ]:
#TF_IDF
documents = {
    "the krishna is good",
    "the krishna is excellent",
    "the krishna is outstaning"
}

#preprocess(tokenize)
import math
from collections import Counter
tokenized_doc = [doc.split() for doc in documents]
N = len(tokenized_doc)
tokenized_doc


[['the', 'krishna', 'is', 'excellent'],
 ['the', 'krishna', 'is', 'outstaning'],
 ['the', 'krishna', 'is', 'good']]

In [ ]:
#compute idf
idf = {} #dict
all_words = set(word for doc in tokenized_doc for word in doc)
for word in all_words:
    df = sum(1 for doc in tokenized_doc if word in doc) #ek word kitne doc mein present hai
    idf[word] = math.log(N/df)
idf


{'krishna': 0.0,
 'the': 0.0,
 'outstaning': 1.0986122886681098,
 'good': 1.0986122886681098,
 'is': 0.0,
 'excellent': 1.0986122886681098}

In [ ]:
#compute tf-idf
tf_idf_vectors = []
for doc in tokenized_doc:
    tf = Counter(doc) #term freq nikl gyi
    doc_len = len(doc)
    tf_idf = {}
    for word in tf:
        tf_value =  tf[word]/doc_len
        tf_idf[word] = tf_value*idf[word]
    tf_idf_vectors.append(tf_idf)
tf_idf_vectors
    



[{'the': 0.0, 'krishna': 0.0, 'is': 0.0, 'excellent': 0.27465307216702745},
 {'the': 0.0, 'krishna': 0.0, 'is': 0.0, 'outstaning': 0.27465307216702745},
 {'the': 0.0, 'krishna': 0.0, 'is': 0.0, 'good': 0.27465307216702745}]

In [ ]:
#sklearn implement
from sklearn .feature_extraction.text import TfidfVectorizer
corpus = {
    "the krishna is good",
    "the krishna is very good",
    "the krishna is mind-blowing"
}
vectorizer = TfidfVectorizer(
    ngram_range=(1,2), #unigram+bigram
    stop_words='english'
)
X = vectorizer.fit_transform(corpus)
print(X.toarray())
print(vectorizer.get_feature_names_out())

[[0.47952794 0.         0.28321692 0.         0.47952794 0.47952794
  0.47952794]
 [0.         0.61980538 0.48133417 0.61980538 0.         0.
  0.        ]
 [0.         0.61980538 0.48133417 0.61980538 0.         0.
  0.        ]]
['blowing' 'good' 'krishna' 'krishna good' 'krishna mind' 'mind'
 'mind blowing']


In [ ]:
X_train_tf = vectorizer.fit_transform(X_train)
X_test_tf = vectorizer.transform(X_test)

nb_model.fit(X_train_tf,y_train)
y_pred = nb_model.predict(X_test_tf)
print("Accuracy",accuracy_score(y_pred,y_test))


Accuracy 0.6578125


In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_tf,y_train)
# X_train  → input (question)
# y_train  → output (answer)

y_pred = model.predict(X_test_bow)

print("Accuracy",accuracy_score(y_pred,y_test))


Accuracy 0.8284375


In [ ]:
#word2vec 

%pip install gensim nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.4/24.4 MB 254.7 kB/s  0:01:14m0:00:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [gensim]2m2/3 [gensim]

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>


False

In [ ]:
%pip install gensim




[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
corpus = [
    "Krishna love machine learning",
    "machine learning is very powerful",
    "Krishna love deep learning",
    "deep learning loves data"
]


In [ ]:
from gensim.utils import simple_preprocess
tokenized_docs = [simple_preprocess(sentence) for sentence in corpus]
print(tokenized_docs)

[['krishna', 'love', 'machine', 'learning'], ['machine', 'learning', 'is', 'very', 'powerful'], ['krishna', 'love', 'deep', 'learning'], ['deep', 'learning', 'loves', 'data']]


In [ ]:
#train
from gensim.models import Word2Vec
model = Word2Vec(
    sentences=tokenized_doc,
    vector_size=100,  #size of embedding
    window=5,
    min_count=1, #ignore words with frequency < 1
    sg=1,   # 1 = Skip-Gram, 0 = CBOW
)

In [ ]:
print("krishna" in model.wv)

True


In [ ]:
vector = model.wv["leanin"]
print(len(vector))
print(vector[:10])   # first 10 values


100
[-0.00861969  0.00366574  0.00518988  0.00574194  0.00746692 -0.00616768
  0.00110561  0.00604728 -0.00284005 -0.00617352]


In [ ]:
model.wv.most_similar("krishna")

[('good', 0.06797593086957932),
 ('excellent', 0.009391169995069504),
 ('outstaning', 0.0045030321925878525),
 ('is', -0.010839181020855904),
 ('the', -0.023671647533774376)]

In [ ]:
#Word2Vec
# Word2Vec words ko vectors deta hai jo meaning carry karte hain.



In [ ]:
%pip install gensim



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
corpus = [
    "Krishna loves machine learning",
    "machine learning is powerful"
    "Krishna loves deep learning",
    "deep learning loves data"
]


In [25]:
from gensim.utils import simple_preprocess
tokenized_doc = [simple_preprocess(sentence) for sentence in corpus]
print(tokenized_doc)

[['krishna', 'loves', 'machine', 'learning'], ['machine', 'learning', 'is', 'powerfulkrishna', 'loves', 'deep', 'learning'], ['deep', 'learning', 'loves', 'data']]


In [26]:
# Train Word2Vec Model
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=tokenized_doc,
    vector_size=100,   # size of embedding
    window=5,          # context window
    min_count=1,       # ignore words with freq < 1
    sg=1               # 1 = Skip-Gram, 0 = CBOW
)


In [27]:
vector = model.wv["learning"]
print(len(vector))   # 100
print(vector[:10])   # first 10 values


100
[-0.00053623  0.00023643  0.00510335  0.00900927 -0.00930295 -0.00711681
  0.00645887  0.00897299 -0.00501543 -0.00376337]


In [28]:
print(model.wv.most_similar("learning"))


[('krishna', 0.09291722625494003), ('is', 0.016134675592184067), ('loves', -0.010839175432920456), ('data', -0.027750372886657715), ('deep', -0.05234674736857414), ('powerfulkrishna', -0.05987630411982536), ('machine', -0.1116705909371376)]


In [ ]:
# Word2Vec sirf:
# word → vector

# ML model ko chahiye:
# document → vector

In [ ]:
import numpy as np
def document_vector(doc,model):
    doc_vec = np.zeros(model.vector_size)
    word_count = 0 #keeps the track of how many valid words found in the document
    for word in doc:
        if word in model.wv: #model.wv contains all the words learned by the model
            doc_vec += model.wv[word]
            word_count += 1
        if word_count > 0 :
            doc_vec = doc_vec/word_count
            
        return doc_vec


In [30]:
X = np.array([document_vector(doc)] for doc in tokenized_doc)
print(X.shape)

()
